# Technical Analysis Template: sRoC = WMA + RoC

Follow the trend the using smooth Rate of Change. The smooth function is WMA.

In [ ]:
import qnt.data as qndata
import qnt.stats as qnstats
import qnt.xr_talib as qnxrtalib
import qnt.graph as qngraph
import qnt.forward_looking as qnfl

# Data

In [ ]:
data = qndata.load_data(min_date="2014-01-01", forward_order=True, dims=("time", "field", "asset"))

# Calc output

In [ ]:
SROC_POSITIVE_TREND_LEVEL = 0.0125

wma = qnxrtalib.WMA(data.sel(field='close'), 290)
sroc = qnxrtalib.ROCP(wma, 35)

is_liquid = data.sel(field="is_liquid")
weights = is_liquid.where(sroc > SROC_POSITIVE_TREND_LEVEL)

weights = weights / weights.sum("asset", skipna=True)

output = weights.fillna(0.0)

## Stats and plots

In [ ]:
stat = qnstats.calc_stat(data, output)
display(stat.to_pandas().tail())

# Improvement

In [ ]:
stat_per_asset = qnstats.calc_stat(data, output, max_periods=40, per_asset = True)
improved_output = output.where(
    stat_per_asset.sel(field='sharpe_ratio').rolling(time=40).mean() > 1
)
stat = qnstats.calc_stat(data, improved_output, max_periods=252 * 3)
stat.to_pandas().tail()

# Checks

In [ ]:
# Use the function from 'qnfl' ensures that no forward-looking
# is taking place. 
def strategy():
    """
    it is the same strtegy, but implemented with xarray
    Entire code of strategy calculation is collected here.
    """
    data = qndata.load_data(min_date="2014-01-01", forward_order=True, dims=("time", "field", "asset"))

    SROC_POSITIVE_TREND_LEVEL = 0.0125

    wma = qnxrtalib.WMA(data.sel(field='close'), 290)
    sroc = qnxrtalib.ROCP(wma, 35)

    is_liquid = data.sel(field="is_liquid")
    weights = is_liquid.where(sroc > SROC_POSITIVE_TREND_LEVEL)

    weights = weights / weights.sum("asset", skipna=True)

    output = weights.fillna(0.0)
    
    stat_per_asset = qnstats.calc_stat(data, output, max_periods=40, per_asset = True)
    improved_output = output.where(
        stat_per_asset.sel(field='sharpe_ratio').rolling(time=40).mean() > 1
    )
    
    return improved_output

# This function runs strategy twice on the different periods: 
# the entire data and data the with a cropped last half year.
# After that this function compares outputs. 
# Overlapped outputs must be same.
output_final = qnfl.load_data_calc_output_and_check_forward_looking(strategy)

In [ ]:
# correlation check
# your strategy should not correlate with other strategies before submission
qnstats.print_correlation(output_final, data)

# Save output

In [ ]:
qndata.write_output(output)